In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\marig\OneDrive\Área de Trabalho\curso arbeitsamt\data immersion\achievement 4\07.2023.Instacart Basket Analysis\Data'

In [3]:
# Loading orders products customer dataframe
ords_prods_cust = pd.read_pickle (os.path.join (path, 'Prepared data', 'orders_products_customer_lastversion.pkl'))

In [21]:
ords_prods_cust.head()

,user_id,State,Age,date_joined,n_dependants,fam_status,income,product_id,product_name,department_id,...,price_range_loc,busiest days,busiest_period_of_day,max_order,loyalty_flag,prices_mean_user,price_flag,user_median_days_since_order,mediandays_flag,Region
0,26711,Missouri,48,1/1/2017,3,married,165665,196,Soda,7,...,Mid range product,Busiest days,Average orders,8,New customer,8.205882,Low Spender,19.0,Frequent customer,Midwest
1,26711,Missouri,48,1/1/2017,3,married,165665,196,Soda,7,...,Mid range product,Regularly busy,Most orders,8,New customer,8.205882,Low Spender,19.0,Frequent customer,Midwest
2,26711,Missouri,48,1/1/2017,3,married,165665,196,Soda,7,...,Mid range product,Busiest days,Most orders,8,New customer,8.205882,Low Spender,19.0,Frequent customer,Midwest
3,26711,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,4,...,Low range product,Slowest days,Most orders,8,New customer,8.205882,Low Spender,19.0,Frequent customer,Midwest
4,26711,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,4,...,Low range product,Busiest days,Average orders,8,New customer,8.205882,Low Spender,19.0,Frequent customer,Midwest


# 1. Creating a regional segmentation of the data

In [4]:
#Identifying four region groups by including specified states into each region:

Notheast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [5]:
#Create a flag for identifying which region:

ords_prods_cust .loc[ords_prods_cust ['State'].isin(Notheast), 'Region'] = 'Northeast'
ords_prods_cust .loc[ords_prods_cust ['State'].isin(Midwest), 'Region'] = 'Midwest'
ords_prods_cust .loc[ords_prods_cust ['State'].isin(South), 'Region'] = 'South'
ords_prods_cust .loc[ords_prods_cust ['State'].isin(West), 'Region'] = 'West'

In [18]:
#Review the output
ords_prods_cust ['Region'].value_counts(dropna = False)

South        10099817
West          7765088
Midwest       7107586
Northeast     5356272
Name: Region, dtype: int64

## 1.1Determining if there is an spending diference between US Regions

In [20]:
ords_prods_cust.columns

Index(['user_id', 'State', 'Age', 'date_joined', 'n_dependants', 'fam_status',
       'income', 'product_id', 'product_name', 'department_id', 'prices',
       'order_id', 'order_number', 'orders_day_of_week', 'order_hour_of_day',
       'days_since_prior_order', 'price_range_loc', 'busiest days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'prices_mean_user', 'price_flag', 'user_median_days_since_order',
       'mediandays_flag', 'Region'],
      dtype='object')

In [6]:
# Creating a crosstab between 'Region' and 'Price flag' columns
crosstab = pd.crosstab(ords_prods_cust['Region'], ords_prods_cust['price_flag'], dropna = False)

In [7]:
# Result
crosstab

price_flag,High Spender,Low Spender
Region,,
Midwest,148321,6959265
Northeast,102905,5253367
South,197110,9902707
West,149922,7615166


# 2. Creating an exclusion flag for low-activity customers (>5 orders)

In [8]:
# Flag criteria
ords_prods_cust.loc[ords_prods_cust['max_order'] <= 5, 'level_of_activity'] = 'Low activity customer' 
ords_prods_cust.loc[ords_prods_cust['max_order'] > 5, 'level_of_activity'] = 'High activity customer' 

In [30]:
# looking the new column
ords_prods_cust.head()

,user_id,State,Age,date_joined,n_dependants,fam_status,income,product_id,product_name,department_id,...,busiest days,busiest_period_of_day,max_order,loyalty_flag,prices_mean_user,price_flag,user_median_days_since_order,mediandays_flag,Region,level_of_activity
0,26711,Missouri,48,1/1/2017,3,married,165665,196,Soda,7,...,Busiest days,Average orders,8,New customer,8.205882,Low Spender,19.0,Frequent customer,Midwest,High activity customer
1,26711,Missouri,48,1/1/2017,3,married,165665,196,Soda,7,...,Regularly busy,Most orders,8,New customer,8.205882,Low Spender,19.0,Frequent customer,Midwest,High activity customer
2,26711,Missouri,48,1/1/2017,3,married,165665,196,Soda,7,...,Busiest days,Most orders,8,New customer,8.205882,Low Spender,19.0,Frequent customer,Midwest,High activity customer
3,26711,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,4,...,Slowest days,Most orders,8,New customer,8.205882,Low Spender,19.0,Frequent customer,Midwest,High activity customer
4,26711,Missouri,48,1/1/2017,3,married,165665,6184,Clementines,4,...,Busiest days,Average orders,8,New customer,8.205882,Low Spender,19.0,Frequent customer,Midwest,High activity customer


In [31]:
# Checking the number of rows
ords_prods_cust.shape

(30328763, 27)

In [13]:
# Creating 2 new dataframes, one for the low activity customers, that we can exclude.
df_low_activity = ords_prods_cust [ords_prods_cust ['level_of_activity'] == 'Low activity customer']

In [14]:
df_low_activity.shape

(1660209, 27)

In [44]:
# Checking the max value
df_low_activity ['max_order'].max()

5

In [15]:
# And one for the high activity customers that we can save
df_high_activiy = ords_prods_cust [ords_prods_cust ['level_of_activity'] == 'High activity customer']

In [16]:
df_high_activiy.shape

(28668554, 27)

In [46]:
# Checking min value
df_high_activiy ['max_order'].min()

6

In [18]:
df_high_activiy.columns

Index(['user_id', 'State', 'Age', 'date_joined', 'n_dependants', 'fam_status',
       'income', 'product_id', 'product_name', 'department_id', 'prices',
       'order_id', 'order_number', 'orders_day_of_week', 'order_hour_of_day',
       'days_since_prior_order', 'price_range_loc', 'busiest days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'prices_mean_user', 'price_flag', 'user_median_days_since_order',
       'mediandays_flag', 'Region', 'level_of_activity'],
      dtype='object')

In [17]:
# Exporting both dataframes to use:
df_high_activiy.to_pickle (os.path.join (path, 'Prepared data', 'ords_prods_cust_high.pkl'))

In [19]:
df_low_activity.to_pickle (os.path.join (path, 'Prepared data', 'ords_prods_cust_low.pkl'))

In [ ]:
# Due to memory issues, it continues in the next cleaned notebook.